# Distribution of influences

To get a grip on how much randomness influences the results, we run the same model lots of times and compare the results.

In [1]:
# for Colab, install fin_benefits and unemployment-gym from Github
#!pip install -q git+https://github.com/ajtanskanen/benefits.git  
#!pip install -q git+https://github.com/ajtanskanen/econogym.git
#!pip install -q git+https://github.com/ajtanskanen/lifecycle-rl.git

# and then restart kernel
  
  # For a specific version:
#!pip install tensorflow==1.15
#!pip install stable-baselines==2.8
  
# restart kernel after running pip's

Then load all modules and set parameters for simulations.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.15 valittaa paljon)
# ei taida toimia piilottaminen
import warnings
warnings.filterwarnings('ignore')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# parameters for the simulation
# episode = 51 / 205 timesteps (1y/3m timestep)
pop_size=10_000 # size of the population to be simulated
size1=10_000_000 #0_000 # number of timesteps in phase 1 training (callback not used)
size2=100 #0_000 # number of timesteps in phase 2 training (callback is used to save the best results)
size3=100 # number of timesteps in phase 1 training (callback not used) for policy changes
batch1=1 # size of minibatch in phase 1 as number of episodes
batch2=9_00  # size of minibatch in phase 1 as number of episodes
callback_minsteps=batch2 # how many episodes callback needs 
deterministic=False # use deterministic prediction (True) or probabilitic prediction (False)
mortality=False # include mortality in computations
randomness=True # include externally given, random state-transitions (parental leaves, disability, lay-offs) 
pinkslip=True # include lay-offs at 5 percent level each year
rlmodel='acktr' # use ACKTR algorithm
twostage=False # ajataan kahdessa vaiheessa vai ei
perusmalli='best/malli_perus3'

# Baseline

Lasketaan työllisyysasteet nykymallissa.

In [3]:
cc1=Lifecycle(env='unemployment-v1',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc1.explain()
cc1.run_distrib(n=100,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=perusmalli,plot=True,cont=True,start_from=perusmalli,results='results/distrib_base',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)

No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 20
max_age 70
min_retirementage 63.5
max_retirementage 68.5
ansiopvraha_kesto300 None
ansiopvraha_kesto400 None
ansiopvraha_toe None
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki None
include_pinkslip True
step 0.25

train...
phase 1
batch 1 learning rate 0.125 scaled 0.125




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






training...








---------------------------------
| explained_variance | 0.983    |
| fps                | 1076     |
| nupdates           | 1        |
| policy_entropy     | 0.86     |
| policy_loss        | 0.0246   |
| total_timesteps    | 0        |
| value_loss         | 0.238    |
-----------------------------

---------------------------------
| explained_variance | 0.854    |
| fps                | 1640     |
| nupdates           | 20       |
| policy_entropy     | 0.741    |
| policy_loss        | -0.187   |
| total_timesteps    | 46075    |
| value_loss         | 1.1      |
---------------------------------
---------------------------------
| explained_variance | 0.962    |
| fps                | 1580     |
| nupdates           | 30       |
| policy_entropy     | 0.738    |
| policy_loss        | 0.0825   |
| total_timesteps    | 70325    |
| value_loss         | 0.464    |
---------------------------------
---------------------------------
| explained_variance | 0.973    |
| fps                | 1552     |
| nupdates           | 40       |
| policy_entropy     | 0.724    |
| policy_loss        | -0.124   |
| total_timesteps    | 94575    |
| value_loss         | 0.574    |
---------------------------------
---------------------------------
| explained_variance | 0.982    |
| fps         

---------------------------------
| explained_variance | 0.963    |
| fps                | 1524     |
| nupdates           | 290      |
| policy_entropy     | 0.764    |
| policy_loss        | -0.0423  |
| total_timesteps    | 700825   |
| value_loss         | 0.504    |
---------------------------------
---------------------------------
| explained_variance | 0.955    |
| fps                | 1523     |
| nupdates           | 300      |
| policy_entropy     | 0.804    |
| policy_loss        | -0.167   |
| total_timesteps    | 725075   |
| value_loss         | 0.544    |
---------------------------------
---------------------------------
| explained_variance | 0.961    |
| fps                | 1529     |
| nupdates           | 310      |
| policy_entropy     | 0.821    |
| policy_loss        | 0.05     |
| total_timesteps    | 749325   |
| value_loss         | 0.489    |
---------------------------------
---------------------------------
| explained_variance | 0.963    |
| fps         

---------------------------------
| explained_variance | 0.959    |
| fps                | 1713     |
| nupdates           | 560      |
| policy_entropy     | 0.761    |
| policy_loss        | 0.0111   |
| total_timesteps    | 1355575  |
| value_loss         | 0.501    |
---------------------------------
---------------------------------
| explained_variance | 0.976    |
| fps                | 1718     |
| nupdates           | 570      |
| policy_entropy     | 0.754    |
| policy_loss        | -0.0341  |
| total_timesteps    | 1379825  |
| value_loss         | 0.388    |
---------------------------------
---------------------------------
| explained_variance | 0.98     |
| fps                | 1722     |
| nupdates           | 580      |
| policy_entropy     | 0.78     |
| policy_loss        | 0.026    |
| total_timesteps    | 1404075  |
| value_loss         | 0.282    |
---------------------------------
---------------------------------
| explained_variance | 0.968    |
| fps         

---------------------------------
| explained_variance | 0.97     |
| fps                | 1804     |
| nupdates           | 830      |
| policy_entropy     | 0.804    |
| policy_loss        | -0.0349  |
| total_timesteps    | 2010325  |
| value_loss         | 0.354    |
---------------------------------
---------------------------------
| explained_variance | 0.974    |
| fps                | 1807     |
| nupdates           | 840      |
| policy_entropy     | 0.727    |
| policy_loss        | -0.0261  |
| total_timesteps    | 2034575  |
| value_loss         | 0.335    |
---------------------------------
---------------------------------
| explained_variance | 0.975    |
| fps                | 1809     |
| nupdates           | 850      |
| policy_entropy     | 0.75     |
| policy_loss        | 0.0387   |
| total_timesteps    | 2058825  |
| value_loss         | 0.294    |
---------------------------------
---------------------------------
| explained_variance | 0.947    |
| fps         

...

---------------------------------
| explained_variance | 0.97     |
| fps                | 2000     |
| nupdates           | 1080     |
| policy_entropy     | 0.736    |
| policy_loss        | -0.124   |
| total_timesteps    | 2616575  |
| value_loss         | 0.419    |
---------------------------------
---------------------------------
| explained_variance | 0.968    |
| fps                | 2001     |
| nupdates           | 1090     |
| policy_entropy     | 0.735    |
| policy_loss        | -0.0619  |
| total_timesteps    | 2640825  |
| value_loss         | 0.435    |
---------------------------------
---------------------------------
| explained_variance | 0.989    |
| fps                | 2001     |
| nupdates           | 1100     |
| policy_entropy     | 0.724    |
| policy_loss        | 0.0475   |
| total_timesteps    | 2665075  |
| value_loss         | 0.239    |
---------------------------------
---------------------------------
| explained_variance | 0.984    |
| fps         

---------------------------------
| explained_variance | 0.975    |
| fps                | 1999     |
| nupdates           | 1350     |
| policy_entropy     | 0.821    |
| policy_loss        | -0.129   |
| total_timesteps    | 3271325  |
| value_loss         | 0.379    |
---------------------------------
---------------------------------
| explained_variance | 0.963    |
| fps                | 1999     |
| nupdates           | 1360     |
| policy_entropy     | 0.701    |
| policy_loss        | -0.0317  |
| total_timesteps    | 3295575  |
| value_loss         | 0.329    |
---------------------------------
---------------------------------
| explained_variance | 0.974    |
| fps                | 1999     |
| nupdates           | 1370     |
| policy_entropy     | 0.747    |
| policy_loss        | -0.0597  |
| total_timesteps    | 3319825  |
| value_loss         | 0.302    |
---------------------------------
---------------------------------
| explained_variance | 0.981    |
| fps         

---------------------------------
| explained_variance | 0.956    |
| fps                | 2001     |
| nupdates           | 1620     |
| policy_entropy     | 0.79     |
| policy_loss        | 0.124    |
| total_timesteps    | 3926075  |
| value_loss         | 0.469    |
---------------------------------
---------------------------------
| explained_variance | 0.958    |
| fps                | 2001     |
| nupdates           | 1630     |
| policy_entropy     | 0.869    |
| policy_loss        | -0.273   |
| total_timesteps    | 3950325  |
| value_loss         | 0.486    |
---------------------------------
---------------------------------
| explained_variance | 0.977    |
| fps                | 2000     |
| nupdates           | 1640     |
| policy_entropy     | 0.813    |
| policy_loss        | 0.18     |
| total_timesteps    | 3974575  |
| value_loss         | 0.447    |
---------------------------------
---------------------------------
| explained_variance | 0.975    |
| fps         

---------------------------------
| explained_variance | 0.965    |
| fps                | 2001     |
| nupdates           | 1890     |
| policy_entropy     | 0.774    |
| policy_loss        | -0.147   |
| total_timesteps    | 4580825  |
| value_loss         | 0.498    |
---------------------------------
---------------------------------
| explained_variance | 0.977    |
| fps                | 2001     |
| nupdates           | 1900     |
| policy_entropy     | 0.749    |
| policy_loss        | 0.0373   |
| total_timesteps    | 4605075  |
| value_loss         | 0.347    |
---------------------------------
---------------------------------
| explained_variance | 0.952    |
| fps                | 2001     |
| nupdates           | 1910     |
| policy_entropy     | 0.729    |
| policy_loss        | 0.0296   |
| total_timesteps    | 4629325  |
| value_loss         | 0.378    |
---------------------------------
---------------------------------
| explained_variance | 0.981    |
| fps         

KeyboardInterrupt: 

In [ ]:
cc2=Lifecycle(env='unemployment-v1',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc2.render_distrib(load='results/distrib_base_v',n=17,stats_results='results/distrib_stats')


# Työttömyysputken poisto

Työttömyysputkelle meneminen on usein hyvin suosittua elinkaarimalleissa. Tarkastellaan millainen työllisyysvaikutus on putken poistamisella.

In [ ]:
cc1_putki=Lifecycle(env='unemployment-v1',minimal=False,include_putki=False,mortality=mortality,
                    perustulo=False,randomness=randomness)
cc1_putki.run_distrib(n=5,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=perusmalli,plot=True,cont=True,start_from=perusmalli,results='results/distrib_poisto',
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage)


In [ ]:
cc1_putki.render_distrib(load='results/putki_results')
cc1_putki.compare_simstats('results/putki_results','results/putki_results')                    